In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.7 MB/s eta 0:00:00


In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

# initialize question and context encoders and their tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

# here is a random context and a related question
context = "Artificial intelligence (AI) is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals, which involves consciousness and emotionality. The distinction between the former and the latter categories is often revealed by the acronym chosen. 'Strong' AI is usually labelled as AGI (Artificial General Intelligence) while attempts to emulate 'natural' intelligence have been called ABI (Artificial Biological Intelligence)."
question = "What is artificial intelligence?"

# tokenize the input (both question and context)
question_inputs = question_tokenizer(question, return_tensors='pt')
context_inputs = context_tokenizer(context, return_tensors='pt')

# encode the input to get embeddings
question_emb = question_encoder(**question_inputs).pooler_output
context_emb = context_encoder(**context_inputs).pooler_output

# compute the similarity between the embeddings (we can use cosine similarity here)
similarity = torch.nn.functional.cosine_similarity(question_emb, context_emb)

print(similarity)

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


tensor([0.7218], grad_fn=<SumBackward1>)


In [1]:
similarity

NameError: ignored

In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast
from transformers import AdamW
import torch

In [4]:
# Load the 'squad' dataset
squad_dataset = load_dataset('squad')

# Let's consider a subset for demonstration
squad_dataset = {
    'train': squad_dataset['train'].select(range(1000)),
    'validation': squad_dataset['validation'].select(range(500)),
}

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Load the DPR tokenizers and models
question_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

question_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
ctx_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequ

In [6]:
def prepare_dataset(example):
    # Tokenize questions and contexts
    question = question_tokenizer(example['question'], truncation=True, padding='max_length', max_length=256)
    context = ctx_tokenizer(example['context'], truncation=True, padding='max_length', max_length=256)

    return {
        'question_input_ids': torch.tensor(question['input_ids']),
        'question_attention_mask': torch.tensor(question['attention_mask']),
        'context_input_ids': torch.tensor(context['input_ids']),
        'context_attention_mask': torch.tensor(context['attention_mask']),
        'labels': torch.tensor(0),  # for SQuAD, we don't have labels as in NQ, so just a dummy label
    }

# Prepare datasets
squad_dataset = {
    'train': squad_dataset['train'].map(prepare_dataset),
    'validation': squad_dataset['validation'].map(prepare_dataset),
}

# Convert to PyTorch DataLoader
dataloader = {
    'train': DataLoader(squad_dataset['train'], batch_size=16, shuffle=True),
    'validation': DataLoader(squad_dataset['validation'], batch_size=16, shuffle=True),
}

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [7]:
# Move models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
question_model = question_model.to(device)
ctx_model = ctx_model.to(device)

# Initialize optimizer
optimizer = AdamW(list(question_model.parameters()) + list(ctx_model.parameters()), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
# Training loop
for epoch in range(3):
    for i, batch in enumerate(dataloader['train']):
        # Move the batch tensors to the same device as the models
        ## stack을 사용하는 이유
        ### for i, batch in enumerate(dataloader['train']): batch-> type(dict)이 나옴
        ### 따라서 이를 직접사용하거나 device push불가능
        ### tensor로 변형해줘야함 . batch['question_input_ids'] = > list # 여전히 전달못함
        ### batch['question_input_ids'][0] => torch.Tensor
        ## 즉 torch.stack을 사용하여 batch를 얻고 사용

        question_input_ids = torch.stack(batch['question_input_ids']).to(device)
        question_attention_mask = torch.stack(batch['question_attention_mask']).to(device)
        context_input_ids = torch.stack(batch['context_input_ids']).to(device)
        context_attention_mask = torch.stack(batch['context_attention_mask']).to(device)

        optimizer.zero_grad()

        # forward pass
        question_outputs = question_model(input_ids=question_input_ids, attention_mask=question_attention_mask)
        ctx_outputs = ctx_model(input_ids=context_input_ids, attention_mask=context_attention_mask)

        # Calculate the loss
        loss = ((question_outputs.pooler_output - ctx_outputs.pooler_output)**2).mean()

        print(f"Batch {i}, Loss: {loss.item()}")

        # backward pass
        loss.backward()

        optimizer.step()

    print(f"Epoch {epoch} completed.")

Batch 0, Loss: 0.3261333107948303
Batch 1, Loss: 0.1703128069639206
Batch 2, Loss: 0.14680278301239014
Batch 3, Loss: 0.10483573377132416
Batch 4, Loss: 0.10017474740743637
Batch 5, Loss: 0.08642880618572235
Batch 6, Loss: 0.08010274171829224
Batch 7, Loss: 0.06518030166625977
Batch 8, Loss: 0.0749344676733017
Batch 9, Loss: 0.051463689655065536
Batch 10, Loss: 0.051494207233190536
Batch 11, Loss: 0.044276461005210876
Batch 12, Loss: 0.04224245995283127
Batch 13, Loss: 0.04082857072353363
Batch 14, Loss: 0.03490179777145386
Batch 15, Loss: 0.03417259454727173
Batch 16, Loss: 0.02862517349421978
Batch 17, Loss: 0.02719070017337799
Batch 18, Loss: 0.026854153722524643
Batch 19, Loss: 0.024386562407016754
Batch 20, Loss: 0.025346850976347923
Batch 21, Loss: 0.024913599714636803
Batch 22, Loss: 0.020218588411808014
Batch 23, Loss: 0.019720930606126785
Batch 24, Loss: 0.016782239079475403
Batch 25, Loss: 0.015847504138946533
Batch 26, Loss: 0.017886601388454437
Batch 27, Loss: 0.01598721928